In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)


In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned UV data)
df = pd.read_csv('CdSe UV Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)
    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
35/35 [==============================] - 1s 7ms/step - loss: 0.0990 - val_loss: 0.0648
Epoch 2/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0428 - val_loss: 0.0355
Epoch 3/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0257 - val_loss: 0.0194
Epoch 4/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0200 - val_loss: 0.0115
Epoch 5/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0180 - val_loss: 0.0078
Epoch 6/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0158 - val_loss: 0.0082
Epoch 7/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0147 - val_loss: 0.0056
Epoch 8/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0128 - val_loss: 0.0047
Epoch 9/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0125 - val_loss: 0.0059
Epoch 10/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0112 - val_loss: 0.0036
Epoch 11/

35/35 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0034
Epoch 84/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0030
Epoch 85/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0036
Epoch 86/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0040
Epoch 87/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0036
Epoch 88/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 0.0032
Epoch 89/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0031
Epoch 90/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 0.0041
Epoch 91/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0026 - val_loss: 0.0032
Epoch 92/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0042
Epoch 93/300


35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 246/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0030
Epoch 247/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0035
Epoch 248/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0030
Epoch 249/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0029
Epoch 250/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0030
Epoch 251/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0025
Epoch 252/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0032
Epoch 253/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0019
Epoch 254/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0029
Epoc

35/35 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0012
Epoch 27/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0011
Epoch 28/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0011
Epoch 29/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0011
Epoch 30/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0012
Epoch 31/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0011
Epoch 32/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0011
Epoch 33/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0011
Epoch 34/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0011
Epoch 35/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 0.0011
Epoch 36/300


35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0011
Epoch 189/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0011
Epoch 190/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0011
Epoch 191/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0011
Epoch 192/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 193/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 9.3063e-04
Epoch 194/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 195/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0015
Epoch 196/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0013
Epoch 197/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0013


...

Epoch 108/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0034
Epoch 109/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 110/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 111/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0029
Epoch 112/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0030
Epoch 113/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0030
Epoch 114/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0043
Epoch 115/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0036
Epoch 116/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 117/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0017 - val_los

35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0038
Epoch 190/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0031
Epoch 191/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0026
Epoch 192/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0033
Epoch 193/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0022
Epoch 194/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0029
Epoch 195/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0032
Epoch 196/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0034
Epoch 197/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0031
Epoch 198/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0030
Epoc

35/35 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0020
Epoch 52/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 53/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 54/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 55/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 56/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 57/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 58/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0019
Epoch 59/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 60/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0022
Epoch 61/300


35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 7.0690e-04
Epoch 211/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 8.0074e-04
Epoch 212/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 8.3398e-04
Epoch 213/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 6.7294e-04
Epoch 214/300
35/35 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.2645e-04
Epoch 215/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 7.8966e-04
Epoch 216/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 8.1528e-04
Epoch 217/300
35/35 [==============================] - 0s 2ms/step - loss: 9.8618e-04 - val_loss: 8.8473e-04
Epoch 218/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 7.4074e-04
Epoch 219/300
35/35 [==============================] - 0s 2ms/ste

Epoch 287/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 8.6243e-04
Epoch 288/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 8.5073e-04
Epoch 289/300
35/35 [==============================] - 0s 2ms/step - loss: 8.9401e-04 - val_loss: 7.1587e-04
Epoch 290/300
35/35 [==============================] - 0s 2ms/step - loss: 9.3536e-04 - val_loss: 8.4720e-04
Epoch 291/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 8.9588e-04
Epoch 292/300
35/35 [==============================] - 0s 1ms/step - loss: 9.1904e-04 - val_loss: 7.0838e-04
Epoch 293/300
35/35 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 7.8190e-04
Epoch 294/300
35/35 [==============================] - 0s 2ms/step - loss: 9.9150e-04 - val_loss: 9.5769e-04
Epoch 295/300
35/35 [==============================] - 0s 1ms/step - loss: 8.7199e-04 - val_loss: 9.0952e-04
Epoch 296/300
35/35 [==============

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0506 0.0095
R2 (eV):	 0.8620 0.0639
RAE (eV):	 0.3330 0.0950
RMSE (eV):	 0.0656 0.0115
MAE (nm):	 10.4159 2.0092
R2 (nm):	 0.8561 0.0810
RAE (nm):	 0.3354 0.1027
RMSE (nm):	 13.4729 2.5911
Absorbance FWHM (direct)
MAE (eV): 	 0.0115 0.0038
R2 (eV):	 0.3973 0.3105
RAE (eV):	 0.6046 0.1609
RMSE (eV):	 0.0185 0.0070
MAE (nm):	 3.0639 0.8792
R2 (nm):	 0.3707 0.2676
RAE (nm):	 0.6360 0.1335
RMSE (nm):	 4.5995 1.6404
Absorbance FWHM (cascade)
MAE (eV): 	 0.0106 0.0028
R2 (eV):	 0.5349 0.1974
RAE (eV):	 0.5570 0.1029
RMSE (eV):	 0.0164 0.0059
MAE (nm):	 2.7324 0.8117
R2 (nm):	 0.5442 0.1886
RAE (nm):	 0.5635 0.1175
RMSE (nm):	 3.9376 1.4415


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0496 0.0092
R2 (eV):	 0.8646 0.0647
RAE (eV):	 0.3269 0.0955
RMSE (eV):	 0.0649 0.0110
MAE (nm):	 10.1859 1.9815
R2 (nm):	 0.8592 0.0824
RAE (nm):	 0.3286 0.1037
RMSE (nm):	 13.2960 2.5126
Absorbance FWHM (direct)
MAE (eV): 	 0.0114 0.0038
R2 (eV):	 0.3997 0.3061
RAE (eV):	 0.5974 0.1606
RMSE (eV):	 0.0184 0.0070
MAE (nm):	 3.0539 0.8445
R2 (nm):	 0.3703 0.2628
RAE (nm):	 0.6349 0.1317
RMSE (nm):	 4.5998 1.6188
Absorbance FWHM (cascade)
MAE (eV): 	 0.0107 0.0028
R2 (eV):	 0.5344 0.1954
RAE (eV):	 0.5621 0.1053
RMSE (eV):	 0.0164 0.0058
MAE (nm):	 2.7708 0.8141
R2 (nm):	 0.5419 0.1860
RAE (nm):	 0.5720 0.1180
RMSE (nm):	 3.9462 1.4245


In [12]:
np.savetxt("NN+MMCN_UV_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_UV_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_UV_Val_cascade.csv", matrix_cascade, delimiter=",")